In [44]:
import numpy as np
import seaborn as sns
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
from pydataset import data
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import env 
import acquire

In [2]:
# pd.set_option('display.max_columns')
# ValueError: Must provide an even number of non-keyword arguments

In [3]:
pd.set_option('display.max_columns', None)

telco = pd.read_csv("telco_churn.csv")

sleep = data('sleepstudy')

mpg = data('mpg')

data(show_doc = True);

# Data Preparation

### IRIS

Using the Iris Data:

    Use the function defined in acquire.py to load the iris data.

    Drop the species_id and measurement_id columns.

    Rename the species_name column to just species.

    Create dummy variables of the species name and concatenate onto the iris dataframe. (This is for practice, we don't always have to encode the target, but if we used species as a feature, we would need to encode it).

    Create a function named prep_iris that accepts the untransformed iris data, and returns the data with the transformations above applied.


In [4]:
iris = acquire.get_iris_data()

In [5]:
def prep_iris(df):
    df.drop(columns = ['species_id', 'measurement_id', 'Unnamed: 0'], inplace = True)
    df.rename(columns={"species_name": "species"}, inplace = True)
    dummy_df = pd.get_dummies(df['species'], dummy_na= False)
    df = pd.concat([df, dummy_df], axis=1)
    return df

In [6]:
iris = prep_iris(iris)

In [7]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species,setosa,versicolor,virginica
0,5.1,3.5,1.4,0.2,setosa,1,0,0
1,4.9,3.0,1.4,0.2,setosa,1,0,0
2,4.7,3.2,1.3,0.2,setosa,1,0,0
3,4.6,3.1,1.5,0.2,setosa,1,0,0
4,5.0,3.6,1.4,0.2,setosa,1,0,0


### Titanic

In [143]:
df = acquire.get_titanic_data()

In [144]:
def impute_mode(df):
    '''
replace non-existant values before breaking it down into training sets
    '''
    imputer = SimpleImputer(missing_values = np.nan, strategy='most_frequent')
    df[['embark_town']] = imputer.fit_transform(df[['embark_town']])
    return df

In [145]:
def split_titanic_data(df):
    '''
    This function performs split on telco data, stratify churn.
    Returns train, validate, and test dfs.
    '''
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123, 
                                        stratify=df.survived)
    train, validate = train_test_split(train_validate, test_size=.2, 
                                   random_state=123, 
                                   stratify=train_validate.survived)
    return train, validate, test

In [146]:
def prep_titanic(df):
    df.drop(columns = ['Unnamed: 0', 'passenger_id', 'deck', 'embarked'], inplace = True)
    dummy_df = pd.get_dummies(df[['sex', 'embark_town', 'class']], dummy_na=False, drop_first= True)
    df = pd.concat([df, dummy_df], axis=1)
    df = impute_mode(df)
    return df

train, validate, test = split_titanic_data(df)
    
    

In [147]:
titanic = prep_titanic(df)

In [148]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   survived                 891 non-null    int64  
 1   pclass                   891 non-null    int64  
 2   sex                      891 non-null    object 
 3   age                      714 non-null    float64
 4   sibsp                    891 non-null    int64  
 5   parch                    891 non-null    int64  
 6   fare                     891 non-null    float64
 7   class                    891 non-null    object 
 8   embark_town              891 non-null    object 
 9   alone                    891 non-null    int64  
 10  sex_male                 891 non-null    uint8  
 11  embark_town_Queenstown   891 non-null    uint8  
 12  embark_town_Southampton  891 non-null    uint8  
 13  class_Second             891 non-null    uint8  
 14  class_Third              8

In [149]:
titanic.shape

(891, 15)

In [150]:
train.shape

(569, 14)

In [151]:
test.shape

(179, 14)

In [152]:
validate.shape

(143, 14)

In [ ]:
#learnign imputer stuff

In [ ]:
#imputer = SimpleImputer(strategy='most_frequent')

In [ ]:
#imputer = imputer.fit(train[['embark_town']])

In [ ]:
#df[['embark_town']] = imputer.transform(df[['embark_town']])

# Telco

In [71]:
df = acquire.get_telco_data()

In [72]:
def split_telco_data(df):
    '''
    This function performs split on telco data, stratify churn.
    Returns train, validate, and test dfs.
    '''
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123, 
                                        stratify=df.churn)
    train, validate = train_test_split(train_validate, test_size=.2, 
                                   random_state=123, 
                                   stratify=train_validate.churn)
    return train, validate, test

In [73]:
def prep_telco(df):
    df.drop(columns = ['Unnamed: 0', 'payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id'], inplace = True)
    dummy_df = pd.get_dummies(df[['contract_type', 'internet_service_type', 'payment_type']], dummy_na=False, drop_first= True)
    df = pd.concat([df, dummy_df], axis=1)
    df = df[df.total_charges != ' ']
    df.total_charges = df.total_charges.astype(float)
    
    
    # encode binary categorical variables into numeric values
    df['gender_encoded'] = df.gender.map({'Female': 1, 'Male': 0})
    df['partner_encoded'] = df.partner.map({'Yes': 1, 'No': 0})
    df['dependents_encoded'] = df.dependents.map({'Yes': 1, 'No': 0})
    df['phone_service_encoded'] = df.phone_service.map({'Yes': 1, 'No': 0})
    df['paperless_billing_encoded'] = df.paperless_billing.map({'Yes': 1, 'No': 0})
    df['churn_encoded'] = df.churn.map({'Yes': 1, 'No': 0})
    return df

train, validate, test = split_telco_data(df)

In [75]:
prep_telco(df)

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check,gender_encoded,partner_encoded,dependents_encoded,phone_service_encoded,paperless_billing_encoded,churn_encoded
0,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.60,593.30,No,One year,DSL,Mailed check,1,0,0,0,0,0,1,1,1,1,1,1,0
1,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.90,542.40,No,Month-to-month,DSL,Mailed check,0,0,0,0,0,0,1,0,0,0,1,0,0
2,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.90,280.85,Yes,Month-to-month,Fiber optic,Electronic check,0,0,1,0,0,1,0,0,0,0,1,1,1
3,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.00,1237.85,Yes,Month-to-month,Fiber optic,Electronic check,0,0,1,0,0,1,0,0,1,0,1,1,1
4,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.90,267.40,Yes,Month-to-month,Fiber optic,Mailed check,0,0,1,0,0,0,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Female,0,No,No,13,Yes,No,Yes,No,No,Yes,No,No,No,55.15,742.90,No,One year,DSL,Mailed check,1,0,0,0,0,0,1,1,0,0,1,0,0
7039,Male,0,Yes,No,22,Yes,Yes,No,No,No,No,No,Yes,Yes,85.10,1873.70,Yes,Month-to-month,Fiber optic,Electronic check,0,0,1,0,0,1,0,0,1,0,1,1,1
7040,Male,0,No,No,2,Yes,No,No,Yes,No,No,No,No,Yes,50.30,92.75,No,Month-to-month,DSL,Mailed check,0,0,0,0,0,0,1,0,0,0,1,1,0
7041,Male,0,Yes,Yes,67,Yes,No,Yes,No,Yes,Yes,No,Yes,No,67.85,4627.65,No,Two year,DSL,Mailed check,0,1,0,0,0,0,1,0,1,1,1,0,0


In [76]:
telco.shape

(7032, 33)

In [77]:
train.shape

(4507, 25)

In [78]:
validate.shape

(1127, 25)

In [79]:
test.shape

(1409, 25)

In [33]:
telco.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check,senior_citizen_encoded,gender_encoded,partner_encoded,dependents_encoded,phone_service_encoded,paperless_billing_encoded,churn_encoded
0,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.30,No,One year,DSL,Mailed check,1,0,0,0,0,0,1,NaN,1,1,1,1,1,0
1,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.40,No,Month-to-month,DSL,Mailed check,0,0,0,0,0,0,1,NaN,0,0,0,1,0,0
2,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check,0,0,1,0,0,1,0,NaN,0,0,0,1,1,1
3,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check,0,0,1,0,0,1,0,NaN,0,1,0,1,1,1
4,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.9,267.40,Yes,Month-to-month,Fiber optic,Mailed check,0,0,1,0,0,0,1,NaN,1,1,0,1,1,1


In [14]:
telco.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 27 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   gender                                7032 non-null   object 
 1   senior_citizen                        7032 non-null   int64  
 2   partner                               7032 non-null   object 
 3   dependents                            7032 non-null   object 
 4   tenure                                7032 non-null   int64  
 5   phone_service                         7032 non-null   object 
 6   multiple_lines                        7032 non-null   object 
 7   online_security                       7032 non-null   object 
 8   online_backup                         7032 non-null   object 
 9   device_protection                     7032 non-null   object 
 10  tech_support                          7032 non-null   object 
 11  streaming_tv     

In [15]:
#indivudual steps

In [16]:
df.loc[df.total_charges == ' ', 'total_charges'] = df.monthly_charges

In [17]:
df.total_charges = df.total_charges.astype('float')

In [18]:
df.total_charges.sort_values()

2060      18.80
6560      18.85
6350      18.85
7033      18.90
981       19.00
         ...   
6275    8564.75
6892    8594.40
6855    8670.10
5360    8672.45
2003    8684.80
Name: total_charges, Length: 7043, dtype: float64

In [19]:
# several of the accounts have no totals

In [20]:
df[df.total_charges == ' '];

In [21]:
#it appears that if they are new cosutomers, they dont have totals till after they pay

In [22]:
df[df.total_charges == ' '];

In [23]:
df.loc[df.total_charges == ' ', 'total_charges'] = df.monthly_charges

In [24]:
df[df.tenure == 0];

In [25]:
df.total_charges = df.total_charges.astype('float')

In [26]:
df = df[df.total_charges != ' ']

In [27]:
# def telco_clean_monthly_total (df):
#     if df.tenure == 0:
#         df.total_charges = df.monthly_charges
#     return df

# telco_clean_monthly_total(df)